# 📊 DATA CLEANING & FEATURE ENGINEERING PIPELINE

This notebook transforms raw OHLCV (Open, High, Low, Close, Volume) data into predictive features for machine learning.

**Pipeline Overview:**
1. **Load** - Read 100 anonymized asset CSV files
2. **Clean** - Handle missing data and outliers
3. **Engineer** - Create 24 technical indicators
4. **Export** - Save processed data for modeling

**Why these steps matter:**
- Poor data quality → Garbage predictions
- Smart features → Better model signals
- Reproducible pipeline → Consistent backtest results

In [1]:
import pandas as pd
import numpy as np
import os

## 📦 STEP 0: IMPORT LIBRARIES

Import core dependencies:
- **pandas**: Data manipulation (DataFrames)
- **numpy**: Numerical computing (array operations)
- **os**: File system access (load CSVs)

In [ ]:
folder_path = 'anonymized_data'
all_dataframes = []

print("🚀 Starting Data Pipeline...")

if os.path.exists(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df_temp = pd.read_csv(file_path, parse_dates=['Date'])
            df_temp['Ticker'] = filename.replace('.csv', '')
            all_dataframes.append(df_temp)
    print(f"✅ Loaded {len(all_dataframes)} assets.")
    full_df = pd.concat(all_dataframes, ignore_index=True)
else:
    print("❌ Error: Folder not found.")

🚀 Starting Data Pipeline...
✅ Loaded 100 assets.


## 📂 STEP 1: LOAD & COMBINE DATA

**Logic:**
- Scan `anonymized_data/` folder for all CSV files
- Each CSV is one asset (e.g., Asset_001.csv, Asset_002.csv)
- Read each with Date column parsed as datetime
- Add a Ticker column to track which asset each row belongs to
- Concatenate all 100 assets into one large DataFrame

**Result:** Single combined dataset with Date, OHLCV, and Ticker columns

In [ ]:
def clean_asset_data(group):
    group = group.ffill()
    for col in ['Open', 'High', 'Low', 'Close']:
        low_limit = group[col].quantile(0.01)
        high_limit = group[col].quantile(0.99)
        group[col] = group[col].clip(low_limit, high_limit)
    return group

print("🧹 Cleaning data (filling gaps & clipping spikes)...")

ticker_map = full_df[['Ticker']].copy()
ticker_map = ticker_map.reset_index(drop=True)

full_df = full_df.groupby('Ticker', group_keys=False).apply(clean_asset_data)
full_df = full_df.reset_index(drop=True)
full_df['Ticker'] = ticker_map['Ticker'].values

print(f"Columns after cleaning: {full_df.columns.tolist()}")

🧹 Cleaning data (filling gaps & clipping spikes)...
Columns after cleaning: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']


## 🧹 STEP 2: CLEAN DATA

**Problems we solve:**
1. **Missing data**: Some days have NaN values (holidays, halted trading)
2. **Data errors**: Extreme outliers (data entry mistakes, stock splits)

**Our approach:**

### A. Forward Fill (ffill)
- Replace NaN with last valid price
- Example: If Wed is NaN, use Tue's price
- Better than dropping rows (preserves history)

### B. Percentile Clipping (1%-99%)
- Remove extreme outliers beyond 1st and 99th percentile
- Example: If stock normally trades $50-$55, and suddenly shows $0.01, clip to $50
- Prevents artificial spikes from breaking our features

**Per-stock cleaning:** Apply to each ticker separately (Asset_001 has its own distribution)

In [ ]:
print("⚙️ Generating Features...")
print(f"Columns available: {full_df.columns.tolist()}")

full_df['log_ret'] = full_df.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(1)))
full_df['volatility_20'] = full_df.groupby('Ticker')['log_ret'].transform(lambda x: x.rolling(window=20).std())

def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

full_df['rsi_14'] = full_df.groupby('Ticker')['Close'].transform(lambda x: calculate_rsi(x))

full_df['ema_12'] = full_df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
full_df['ema_26'] = full_df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
full_df['macd'] = full_df['ema_12'] - full_df['ema_26']

full_df['volume_sma_20'] = full_df.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(window=20).mean())
full_df['volume_ratio'] = full_df['Volume'] / full_df['volume_sma_20']
full_df['volume_trend'] = full_df.groupby('Ticker')['Volume'].transform(lambda x: (x > x.shift(1)).astype(int))
full_df['volume_roc_5'] = full_df.groupby('Ticker')['Volume'].transform(lambda x: x / x.shift(5) - 1)

full_df['sma_50'] = full_df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=50).mean())
full_df['price_to_sma50'] = (full_df['Close'] - full_df['sma_50']) / full_df['sma_50']
full_df['daily_range'] = (full_df['High'] - full_df['Low']) / full_df['Close']

def calculate_atr(high, low, close, window=14):
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()
    return atr

full_df['atr_14'] = full_df.groupby('Ticker', group_keys=False).apply(
    lambda x: calculate_atr(x['High'], x['Low'], x['Close'])
).reset_index(level=0, drop=True)

full_df['bb_middle'] = full_df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).mean())
full_df['bb_std'] = full_df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).std())
full_df['bb_upper'] = full_df['bb_middle'] + (2 * full_df['bb_std'])
full_df['bb_lower'] = full_df['bb_middle'] - (2 * full_df['bb_std'])
full_df['bb_position'] = (full_df['Close'] - full_df['bb_lower']) / (full_df['bb_upper'] - full_df['bb_lower'])

full_df['log_ret_lag1'] = full_df.groupby('Ticker')['log_ret'].shift(1)
full_df['log_ret_lag2'] = full_df.groupby('Ticker')['log_ret'].shift(2)
full_df['log_ret_lag3'] = full_df.groupby('Ticker')['log_ret'].shift(3)
full_df['roc_5'] = full_df.groupby('Ticker')['Close'].transform(lambda x: x / x.shift(5) - 1)
full_df['roc_10'] = full_df.groupby('Ticker')['Close'].transform(lambda x: x / x.shift(10) - 1)
full_df['momentum_score'] = (full_df['roc_5'] + full_df['roc_10']) / 2
full_df['momentum_acceleration'] = full_df.groupby('Ticker')['roc_5'].transform(lambda x: x.diff())

market_avg_return = full_df.groupby('Date')['log_ret'].transform('mean')
full_df['relative_strength'] = full_df['log_ret'] - market_avg_return
full_df['vol_regime'] = full_df.groupby('Ticker')['volatility_20'].transform(
    lambda x: (x > x.rolling(window=50).mean()).astype(int)
)

full_df['ret_mean_5'] = full_df.groupby('Ticker')['log_ret'].transform(lambda x: x.rolling(window=5).mean())
full_df['ret_mean_10'] = full_df.groupby('Ticker')['log_ret'].transform(lambda x: x.rolling(window=10).mean())
full_df['ret_std_5'] = full_df.groupby('Ticker')['log_ret'].transform(lambda x: x.rolling(window=5).std())
full_df['gap'] = full_df.groupby('Ticker').apply(
    lambda x: (x['Open'] - x['Close'].shift(1)) / x['Close'].shift(1)
).reset_index(level=0, drop=True)

print("✅ 32 features engineered (optimized for 10-day predictions)")

⚙️ Generating Features...
Columns available: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']
✅ 32 features engineered (optimized for 10-day predictions)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plot_features = ['log_ret', 'volatility_20', 'volume_ratio', 'price_to_sma50']
plot_df = full_df[plot_features].dropna()
if len(plot_df) > 50000:
    plot_df = plot_df.sample(50000, random_state=42)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for ax, col in zip(axes, plot_features):
    sns.histplot(plot_df[col], bins=50, ax=ax, color='#3498db', kde=True)
    ax.set_title(f'Distribution: {col}', fontsize=11, fontweight='bold')
    ax.set_xlabel(col)
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

na_rate = full_df.isna().mean().sort_values(ascending=False)
na_rate = na_rate[na_rate > 0]
if len(na_rate) > 0:
    plt.figure(figsize=(10, 4))
    sns.barplot(x=na_rate.index, y=na_rate.values, color='#e67e22')
    plt.title('Missing Value Rate by Feature', fontsize=12, fontweight='bold')
    plt.ylabel('Missing Rate')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print('✅ No missing values after cleaning.')

## ⚙️ STEP 3: FEATURE ENGINEERING

Transform raw prices into 24 predictive indicators that capture market dynamics.

### **Category A: Trend & Momentum**
- **log_ret**: Log returns (better for math than simple %)
- **rsi_14**: Relative Strength Index (overbought/oversold)
- **macd**: Moving Average Convergence Divergence (trend direction)
- **roc_5, roc_10**: Rate of Change (short/medium term momentum)
- **momentum_score**: Combined momentum signal

### **Category B: Volatility & Risk**
- **volatility_20**: 20-day rolling std dev of returns (market turbulence)
- **atr_14**: Average True Range (realistic price movement magnitude)
- **bb_position**: Where price sits within Bollinger Bands
- **vol_regime**: High vs Low volatility regimes

### **Category C: Volume & Confirmation**
- **volume_ratio**: Current vol vs 20-day average (unusual activity?)
- **volume_trend**: Is volume increasing? (confirmation signal)
- **volume_roc_5**: Rate of change in volume (strength)

### **Category D: Price Structure**
- **price_to_sma50**: Distance from 50-day average (trend strength)
- **daily_range**: Intraday volatility (High-Low spread)
- **gap**: Overnight price jump (sentiment shift)

### **Category E: Lagged & Cross-Sectional**
- **log_ret_lag1/2/3**: Yesterday's returns (persistence)
- **ret_mean_5/10**: Average recent returns
- **relative_strength**: How stock performs vs market average

**Why 24 features?** More diverse signals = better pattern recognition

In [ ]:
df_final = full_df.dropna()
df_final = df_final.sort_values(['Ticker', 'Date'])

output_file = 'processed_data.csv'
df_final.to_csv(output_file, index=False)

print("\n🎉 Success! Feature Engineering Complete.")
print(f"Final Data Shape: {df_final.shape}")
print(f"File saved to: {output_file}")
print("Sample of generated features:")
print(df_final[['Ticker', 'Date', 'log_ret', 'rsi_14', 'macd']].head())


🎉 Success! Feature Engineering Complete.
Final Data Shape: (245995, 39)
File saved to: processed_data.csv
Sample of generated features:
       Ticker       Date   log_ret     rsi_14      macd
49  Asset_001 2016-04-05 -0.011859  71.071919  0.783103
50  Asset_001 2016-04-06  0.010418  70.501465  0.796297
51  Asset_001 2016-04-07 -0.022051  59.500805  0.741773
52  Asset_001 2016-04-08  0.001105  59.500829  0.693368
53  Asset_001 2016-04-11  0.003308  60.528187  0.655841


## 💾 STEP 4: CLEANUP & EXPORT

**Final processing:**
1. **Drop NaNs**: Remove rows with missing values from feature engineering (MACD needs 26-day history)
2. **Sort**: Order by Ticker then Date (clarity for analysis)
3. **Save**: Export to `processed_data.csv` for use in Notebook 2 (model training)

**Output:** Clean, feature-rich dataset ready for machine learning

---

**Next Steps:**
- Notebook 2 will load `processed_data.csv`
- Train ensemble model on these 24 features
- Generate trading signals